In [61]:
import pandas as pd
import os
import tarfile
from PIL import Image
import torchvision 
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset
from datetime import datetime

In [82]:
#Reading solar images
solar_Images=[f for f in os.listdir("img") if f.endswith(".jpg")]

#Loading images
transformed_Images=[]
for imageFilePath in solar_Images:
    image=Image.open("img\\"+imageFilePath)
    tensor_Transform=torchvision.transforms.ToTensor()
    transformed_Image=tensor_Transform(image)
    transformed_Images.append(transformed_Image)
    
images_Dataset=TensorDataset(transformed_Image)

In [92]:
#Reading flare dataset.
flareDataset=pd.read_csv('flareDataset.csv')
num_Rows, num_Columns=flareDataset.shape

#Setting everything to a universal time scale, with the 0 of time being the first image. 
initial_Time_Split=solar_Images[0].split("_")
date=initial_Time_Split[0]
time=initial_Time_Split[1]
s=""
s+=(f"{date[:4]}-{date[4:6]}-{date[6:]}")
s+=" "
s+=f"{time[:2]}:{time[2:4]}:{time[4:]}"
t0=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')

deltat_Images=[]

for imageName in solar_Images:
    s_split=imageName.split("_")
    s="" 
    date=s_split[0]
    time=s_split[1]
    s+=(f"{date[:4]}-{date[4:6]}-{date[6:]}")
    s+=" "
    s+=f"{time[:2]}:{time[2:4]}:{time[4:]}"
    t=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    deltat_Images.append((t-t0).total_seconds()/3600)
    
deltat_Flares=[]
for i in range(num_Rows):
    s_split=flareDataset.loc[i, "T_REC"].split(" ")
    s="" 
    date=s_split[0]
    time=s_split[1]
    s+=date
    s+=" "
    s+=time
    t=datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    flareDataset.loc[i, "T_REC"]=(t-t0).total_seconds()/3600
    
for i in range(num_Rows):
    if int(flareDataset.loc[i,"T_REC"])<0 or int(flareDataset.loc[i, "FlareNumber"]) !=1:
        flareDataset.drop(i, inplace=True)
   
flareDataset.reset_index(drop=True, inplace=True)

num_Rows, num_Columns=flareDataset.shape

#Normalizing flare database for gradient descent, and dropping any values that will not be useful.
flareDataset.drop("FlareNumber", axis=1, inplace=True)
flareDataset.drop("NOAA_AR", axis=1, inplace=True)
flareDataset.drop("QUALITY", axis=1, inplace=True)
flareDataset.drop("MEANPOT", axis=1, inplace=True)

means=[]
stds=[]

for col in flareDataset:
    if col!="T_REC":
        print(flareDataset[col])
        mean=flareDataset[col].mean()
        std=flareDataset[col].std()
        means.append(mean)
        stds.append(std)
        for i in range(num_Rows):
            flareDataset.loc[i, col]-=mean
            flareDataset.loc[i, col]/=std
        

0      45
1      15
2      11
3       3
4      14
       ..
375    58
376    58
377    70
378    70
379    70
Name: Longitude, Length: 380, dtype: int64
0      12
1      19
2      19
3      20
4      21
       ..
375     9
376     9
377     9
378     9
379     9
Name: Latitude, Length: 380, dtype: int64
0       956.465
1       837.486
2       488.687
3      2624.992
4      2887.519
         ...   
375    5769.701
376    5522.014
377    5562.046
378    5072.454
379    4000.937
Name: TOTUSJH, Length: 380, dtype: float64
0      2.291500e+10
1      1.488800e+10
2      7.451400e+09
3      2.934400e+10
4      3.600500e+10
           ...     
375    7.859000e+10
376    7.859300e+10
377    7.901800e+10
378    7.555800e+10
379    7.261200e+10
Name: TOTBSQ, Length: 380, dtype: float64
0      2.264995e+23
1      7.183306e+22
2      6.566911e+22
3      5.750937e+23
4      7.016926e+23
           ...     
375    1.449073e+24
376    1.307202e+24
377    1.324064e+24
378    1.292672e+24
379    1.16288

C:\Users\Shiyan Liu\AppData\Local\Temp\ipykernel_20848\280375503.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '12.573684210526316' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  flareDataset.loc[i, col]-=mean
C:\Users\Shiyan Liu\AppData\Local\Temp\ipykernel_20848\280375503.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-2.1657894736842103' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  flareDataset.loc[i, col]-=mean


0      2.004485e+13
1      1.812281e+13
2      1.097168e+13
3      4.529535e+13
4      4.896081e+13
           ...     
375    8.285670e+13
376    7.869333e+13
377    7.924306e+13
378    7.544420e+13
379    6.169161e+13
Name: TOTUSJZ, Length: 380, dtype: float64
0       138.408
1        14.549
2         3.039
3       737.994
4       764.512
         ...   
375    2579.513
376    1810.464
377    1588.055
378    1190.330
379     807.002
Name: ABSNJZH, Length: 380, dtype: float64
0      2.368424e+12
1      7.678419e+12
2      1.142786e+12
3      2.036685e+13
4      1.275279e+13
           ...     
375    7.062008e+13
376    4.244859e+13
377    3.736212e+13
378    2.599231e+13
379    1.945459e+13
Name: SAVNCPP, Length: 380, dtype: float64
0      1.628852e+22
1      1.080872e+22
2      5.380702e+21
3      1.863133e+22
4      2.222252e+22
           ...     
375    5.622430e+22
376    5.956309e+22
377    5.949862e+22
378    5.917355e+22
379    6.010767e+22
Name: USFLUX, Length: 380, dtype: f

In [59]:
#Residual neural network for regression.

weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1
resnet=torchvision.models.resnet18(weights)

#Replacing classification layer with regression layer.
resnet.fc=nn.Linear(512, 1, bias=False)




C:\Users\Shiyan Liu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
